In [ ]:
# TODO: implement/test eval functionality for new encoding here, ONE PIECE AT A TIME
# (bring pieces over from eval notebook as needed...)

# leaf category should be the only output currently...START HERE!!

# TODO - use NEW_ENCODING astera dataset and NEW_ENCODING dragom model
    # --> will have to recreate the DRAGON model each time I add a new output...
# TODO - test BATCHES
# TODO - add the rest of the outputs one by one...
# TODO - put each individual predicted output in a column (so we can compute individual accuracies)
# TODO - once all outputs present, convert to a data type prediction (type_seq string?)
#        and evaluate accuracy of the whole prediction
